In [1]:
import json
import pandas as pd
import numpy as np

In [211]:
A = 0.6
a = 3.5e4 # [price]
b = 3.5 # [-]
B = 1
c = 2e5 # [games]
d = 0   # [-]

version_list = ["all_nif", "all_ifs"]
position_list = ["GK", "RB,RWB", "LB,LWB", "CB", "CDM,CM,CAM", "RM,RW,RF", "LM,LW,LF", "CF,ST"]
time_list = ["2019120723", "2019120816"]

In [209]:
def import_scraped_data(time, position, version):
    filename = "../scraped_data/%s_%s_%s.json" % (time, position, version)
    with open(filename, encoding='utf-8') as json_data:
        d = json.load(json_data)
        json_data.close()
    return d

def quality(games, price):
    P = np.array(price.to_list())
    N = np.array(games.to_list())
    Q = (A + (1 - A) / (1 + np.exp(-(P / a - b)))) * (1 - B * np.exp(-(N / c - d)))
    return Q

def clean_dataframe(df):
    df = df[["Name", "Rating", "PS4", "Games"]]
    df.rename(columns={'PS4':'Price'}, inplace=True)
    df.Name = df.Name.str.replace(r"\(.*\)","")
    df.Rating = pd.to_numeric(df.Rating)
    df.Games = pd.to_numeric(df.Games.str.replace(",",""))
    df.Price = (df.Price.replace(r'[KM]+$', '', regex=True).astype(float) * \
                    df.Price.str.extract(r'[\d\.]+([KM]+)', expand=False)
                    .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))
    df.Price = df.Price.astype("int64")
#     df["Quality"] = quality(df.Games, df.Price)
    return df

def dataframe_position(time, position):
    d1 = import_scraped_data(time, position, version_list[0])
    d2 = import_scraped_data(time, position, version_list[1])
    df1 = clean_dataframe(pd.DataFrame(d1))
    df2 = clean_dataframe(pd.DataFrame(d2))
    frames = [df1, df2]
    df = pd.concat(frames, ignore_index=True)
    df["Quality"] = quality(df.Games, df.Price)
    df = df.sort_values(by=['Quality'], ascending=False)
    return df

def dataframe_position_dt(position):
    df0 = dataframe_position(time_list[0], position)
    df1 = dataframe_position(time_list[1], position)
    dn = np.zeros(len(df1.index))
    i = 0
    for index_df1, row_df1 in df1.iterrows():
        for index_df0, row_df0 in df0.iterrows():
            if row_df1["Name"] == row_df0["Name"] and row_df1["Rating"] == row_df0["Rating"]:
                dn[i] = row_df1["Games"] - row_df0["Games"]
        i += 1
    df1.Games = dn.astype(int)
    df1.Quality = quality(df1.Games, df1.Price)
    df1.sort_values(by=['Quality'], ascending=False, inplace=True)
    return df1

In [227]:
df = dataframe_position_dt(position_list[7])
df.sort_values(by=['Games'], ascending=False, inplace=True)
df.head(60)

c:\users\truls\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
c:\users\truls\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
c:\users\truls\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://

,Name,Rating,Price,Games,Quality
1,Heung Min Son,87,436000,213532,0.656154
3,Kylian Mbappé,89,917000,183393,0.600270
15,Cristiano Ronaldo,93,1210000,132351,0.484055
8,Dries Mertens,87,40500,128781,0.301500
0,Lucas,83,6300,101064,0.243549
31,Karim Benzema,88,60500,100101,0.259162
35,Lautaro Martínez,84,16250,97649,0.238861
17,Robert Lewandowski,89,42250,92674,0.236109
9,Aubameyang,88,123000,92001,0.295502
2,Marcus Rashford,83,8200,82215,0.207195
